# Team Project - Reducer

### Reducer - Libraries & File I/O

In [1]:
# Libraries needed for reducer.
import sys
import string
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt
import json

In [2]:

# Define File I/O for reducer.
finput = open('output files/mapped_flight_data.txt', 'r')
foutput = open('output files/reduced_flight_data.txt','w')

### Reducer - Distance Function

#### Haversine Function

In [3]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees).
    """
    # Convert decimal degrees to radians.
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula. 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Earth radius in kilometers (use 3956 for miles).
    # Determines return value units.
    r = 6371
    return c * r

#### Lat. and Lon. of Beijing

In [4]:
# Latitude and Longitude of Beijing.
# Conversion from coordinates to lat./lon. done with 'fcc.gov'.
# https://www.fcc.gov/media/radio/dms-decimal
beijing_lat = 39.9 # 39°54′N (39 degrees, 54 minutes north)
beijing_lon = 116.4 # 116°24′E (116 degrees, 24 minutes east)

#### Example of Haversine Function Output

In [5]:
# Calculate distance between Mcgill university and Beijing.
mcgill_lat = 45.5041  # 45.5048° N
mcgill_lon = -73.5747  # 73.5772° W

mcgill_beijing = haversine(beijing_lon, beijing_lat, mcgill_lon, mcgill_lat)
print(f"The distance between McGill University and Beijing is {round(mcgill_beijing, 2)} km.")

The distance between McGill University and Beijing is 10466.65 km.


### Reducer - Output

In [6]:
# Function to output summarized data into file output.
def reduceroutput(data):
    dataline = json.dumps(data) + "\n"
    foutput.write(dataline)

### Reducer - Group Values by Key

In [7]:
# Global variables used by 'grouper' function.
previous_key = ""
data_array = []

In [8]:
# Function to group the values by key.
def grouper(line):
    
    global previous_key, data_array
    key, value = line.split("\t", 1)
    
    if (key == previous_key or previous_key == ""):
        data_array.append(json.loads(value))
    else:
        summarize(data_array)
        data_array = [json.loads(value)]
    
    previous_key = key

### Reducer - Summarization

In [9]:
# Function to summarize data including aggregation calculation of distance.
def summarize(data_array):

    dataframe = pd.DataFrame(data_array)
    
    data = {
        "ident": data_array[0]["ident"],
        "id": data_array[0]["id"],
        "distance": haversine(beijing_lon, beijing_lat, float(data_array[len(data_array)-1]["lon"]), float(data_array[len(data_array)-1]["lat"]))
    }
    
    reduceroutput(data)

### Reducer - Input Splitter

In [10]:
for line in finput:
    if (line != ""):
        grouper(line)

In [11]:
if (len(data_array) > 0):
    summarize(data_array)

foutput.close()
finput.close()

### Convert the Reducer Output in Text File to Dataframe

(i.e. "groupassignmentdata_reducerout.txt") to Pandas Dataframe and then to CSV file

In [12]:
# Code block to convert reducer output into dataframe.
with open('output files/reduced_flight_data.txt') as f:
    lines = f.readlines()

data_output = []
for line in lines:
    data_output.append(json.loads(line))

df = pd.DataFrame(data_output)
df

,ident,id,distance
0,SWA5019,SWA5019-1535606759-airline-0441,10759.265140
1,AZA1679,AZA1679-1535606747-airline-0175,8141.319740
2,AFL1125,AFL1125-1535606756-airline-0074,5860.543678
3,ETD401,ETD401-1535606750-airline-0002,3965.285125
4,N358TS,N358TS-1535806288-2-0-195,10213.523989
...,...,...,...
15035,RPA4408,RPA4408-1535606746-airline-0561,10861.028122
15036,SWA3550,SWA3550-1535606759-airline-0668,10902.123514
15037,HDA622,HDA622-1535606752-airline-0316,1191.618044
15038,ASQ4096,ASQ4096-1535606757-airline-0071,10693.310969


### Sort the dataframe with the list of all flights by closest to furthest to Beijing

In [13]:
# Command to sort dataframe by distance in ascending order.
df_sorted = df.sort_values(by=['distance'])
df_sorted

,ident,id,distance
8598,CSN6284,CSN6284-1535606749-airline-0547,19.733078
13275,CSN6284,CSN6284-1535606749-airline-0547,19.736027
10918,CSN8670,CSN8670-1535606749-airline-0070,29.207952
2482,CHH7136,CHH7136-1535606751-airline-0050,30.008014
4215,CSN8670,CSN8670-1535606749-airline-0070,30.087326
...,...,...,...
9859,ANS860,ANS860-1535642100-schedule-0001,19588.829465
4225,ARG1681,ARG1681-1535606747-airline-0211,19645.969261
3447,ARG1554,ARG1554-1535606747-airline-0123,19715.112374
5662,AUT2881,AUT2881-1535606747-airline-0131,19718.507110


### Generate CSV file from the sorted dataframe above

In [14]:
# Command for CSV output of dataframe.
df_sorted.to_csv('output files/flights_sorted_by_distance.csv', index=False)

------------------------------

### Data Analysis

#### [1] Analysis by Flight

In [15]:
# Reset the row index in dataframe for further analysis.
df_sorted = df_sorted.reset_index(drop=True)
df_sorted

,ident,id,distance
0,CSN6284,CSN6284-1535606749-airline-0547,19.733078
1,CSN6284,CSN6284-1535606749-airline-0547,19.736027
2,CSN8670,CSN8670-1535606749-airline-0070,29.207952
3,CHH7136,CHH7136-1535606751-airline-0050,30.008014
4,CSN8670,CSN8670-1535606749-airline-0070,30.087326
...,...,...,...
15035,ANS860,ANS860-1535642100-schedule-0001,19588.829465
15036,ARG1681,ARG1681-1535606747-airline-0211,19645.969261
15037,ARG1554,ARG1554-1535606747-airline-0123,19715.112374
15038,AUT2881,AUT2881-1535606747-airline-0131,19718.507110


In [16]:
# Print statements for Min, Max & Mean of distance data.
print(f"Closest Distance Data: Flight = {df_sorted['ident'][0]} | Distance = {df_sorted['distance'][0]}")
print(f"Farthest Distance Data: Flight = {df_sorted['ident'][9746]} | Distance = {df_sorted['distance'][9746]}")
print(f"Average Distance of All Flights: Total Number of Flights = {len(df_sorted)} | Average Distance = {np.mean(df_sorted['distance'])}")

Closest Distance Data: Flight = CSN6284 | Distance = 19.733078279680058
Farthest Distance Data: Flight = ROU1855 | Distance = 10385.868854392329
Average Distance of All Flights: Total Number of Flights = 15040 | Average Distance = 8448.40570079551


#### [2] Analysis by Airline
* [2-a] Create a table with the following fields:
    > - airlines (the first three letters of flight "ident")
    > - count (count of flights by the same airline)
    > - distance_sum (total distance per airline)
    > - distance_mean (average distance per airline)
* [2-b] Generate a table sorted by distance_mean in ascending order
* [2-c] Generate a table sorted by count in descending order

##### [2-a] Create a table with the following fields

In [17]:
# Prepare dataframe to create table with airline
df_sorted["distance_2"] = df_sorted["distance"] 
df_sorted["count"] = 1
df_sorted.info()
df_sorted

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15040 entries, 0 to 15039
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ident       15040 non-null  object 
 1   id          15040 non-null  object 
 2   distance    15040 non-null  float64
 3   distance_2  15040 non-null  float64
 4   count       15040 non-null  int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 587.6+ KB


,ident,id,distance,distance_2,count
0,CSN6284,CSN6284-1535606749-airline-0547,19.733078,19.733078,1
1,CSN6284,CSN6284-1535606749-airline-0547,19.736027,19.736027,1
2,CSN8670,CSN8670-1535606749-airline-0070,29.207952,29.207952,1
3,CHH7136,CHH7136-1535606751-airline-0050,30.008014,30.008014,1
4,CSN8670,CSN8670-1535606749-airline-0070,30.087326,30.087326,1
...,...,...,...,...,...
15035,ANS860,ANS860-1535642100-schedule-0001,19588.829465,19588.829465,1
15036,ARG1681,ARG1681-1535606747-airline-0211,19645.969261,19645.969261,1
15037,ARG1554,ARG1554-1535606747-airline-0123,19715.112374,19715.112374,1
15038,AUT2881,AUT2881-1535606747-airline-0131,19718.507110,19718.507110,1


In [18]:
# Write a function to get the first three letter of airline from flight data.
def get_airline(string):
    airline = string[:3]
    return airline

In [19]:
# Add 'airline' column by applying 'get_airline' function to column 'ident'.
df_sorted["airline"] = df_sorted["ident"].apply(get_airline)

# Check output.
df_sorted.info()
df_sorted

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15040 entries, 0 to 15039
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ident       15040 non-null  object 
 1   id          15040 non-null  object 
 2   distance    15040 non-null  float64
 3   distance_2  15040 non-null  float64
 4   count       15040 non-null  int64  
 5   airline     15040 non-null  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 705.1+ KB


,ident,id,distance,distance_2,count,airline
0,CSN6284,CSN6284-1535606749-airline-0547,19.733078,19.733078,1,CSN
1,CSN6284,CSN6284-1535606749-airline-0547,19.736027,19.736027,1,CSN
2,CSN8670,CSN8670-1535606749-airline-0070,29.207952,29.207952,1,CSN
3,CHH7136,CHH7136-1535606751-airline-0050,30.008014,30.008014,1,CHH
4,CSN8670,CSN8670-1535606749-airline-0070,30.087326,30.087326,1,CSN
...,...,...,...,...,...,...
15035,ANS860,ANS860-1535642100-schedule-0001,19588.829465,19588.829465,1,ANS
15036,ARG1681,ARG1681-1535606747-airline-0211,19645.969261,19645.969261,1,ARG
15037,ARG1554,ARG1554-1535606747-airline-0123,19715.112374,19715.112374,1,ARG
15038,AUT2881,AUT2881-1535606747-airline-0131,19718.507110,19718.507110,1,AUT


In [20]:
# Generate Table.
table = df_sorted.pivot_table(values=['distance', 'count', 'distance_2'], index=['airline'],
                    aggfunc={'count': np.sum, 'distance_2': np.mean, 'distance': np.sum})

table = table.reset_index()  # reset the index row to change 'airline' from index to column

# Rename the columns.
table = table.rename({'distance': 'distance_sum', 'distance_2': 'distance_mean'}, axis=1)

# Check dataframe.
table.info()
table

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 660 entries, 0 to 659
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   airline        660 non-null    object 
 1   count          660 non-null    int64  
 2   distance_sum   660 non-null    float64
 3   distance_mean  660 non-null    float64
dtypes: float64(2), int64(1), object(1)
memory usage: 20.8+ KB


,airline,count,distance_sum,distance_mean
0,40,3,32824.408950,10941.469650
1,9H8,3,2715.252349,905.084116
2,A07,2,38804.958054,19402.479027
3,AAF,4,35848.899948,8962.224987
4,AAH,1,8057.555237,8057.555237
...,...,...,...,...
655,XLF,5,41978.023427,8395.604685
656,XOJ,11,117896.219338,10717.838122
657,XSR,4,43055.424809,10763.856202
658,YV3,2,25861.930679,12930.965339


#### [2-b] Generate a table sorted by distance_mean in ascending order

In [21]:
# Sort 'table_sorted_distance' dataframe by 'distance_mean'.
table_sorted_distance = table.sort_values(by=['distance_mean'])
table_sorted_distance

,airline,count,distance_sum,distance_mean
228,GDC,5,343.537579,68.707516
159,CUA,9,4549.667494,505.518610
329,LTU,1,520.822957,520.822957
498,QDA,10,6604.206663,660.420666
248,HBH,7,5560.420592,794.345799
...,...,...,...,...
55,ATM,4,75271.850071,18817.962518
60,AUT,14,269486.942019,19249.067287
82,BOV,2,38561.471265,19280.735633
2,A07,2,38804.958054,19402.479027


In [22]:
# Reset the row index in dataframe for furhter analysis.
table_sorted_distance = table_sorted_distance.reset_index(drop=True)
table_sorted_distance

,airline,count,distance_sum,distance_mean
0,GDC,5,343.537579,68.707516
1,CUA,9,4549.667494,505.518610
2,LTU,1,520.822957,520.822957
3,QDA,10,6604.206663,660.420666
4,HBH,7,5560.420592,794.345799
...,...,...,...,...
655,ATM,4,75271.850071,18817.962518
656,AUT,14,269486.942019,19249.067287
657,BOV,2,38561.471265,19280.735633
658,A07,2,38804.958054,19402.479027


In [23]:
print(f"Lowest Distance Mean: Airline = {table_sorted_distance['airline'][0]} | Distance Mean = {table_sorted_distance['distance_mean'][0]}")
print(f"Highest Distance Mean: Airline = {table_sorted_distance['airline'][659]} | Distance Mean = {table_sorted_distance['distance_mean'][659]}")

Lowest Distance Mean: Airline = GDC | Distance Mean = 68.7075158115285
Highest Distance Mean: Airline = ANS | Distance Mean = 19588.829464803355


#### [2-c] Generate a table sorted by count in descending order

In [24]:
table_sorted_count = table.sort_values(by=['count'], ascending=False)
table_sorted_count

,airline,count,distance_sum,distance_mean
5,AAL,721,7.952137e+06,11029.315666
168,DAL,613,6.491973e+06,10590.493908
558,SWA,546,5.940822e+06,10880.627004
606,UAL,491,5.032830e+06,10250.162381
522,RYR,405,3.391457e+06,8373.967171
...,...,...,...,...
552,SPA,1,1.131082e+04,11310.817082
550,SOL,1,7.028465e+03,7028.465472
549,SMR,1,4.666830e+03,4666.829826
462,NTA,1,8.135694e+03,8135.694048


In [25]:
# Reset the row index in dataframe for further analysis.
table_sorted_count = table_sorted_count.reset_index(drop=True)
table_sorted_count

,airline,count,distance_sum,distance_mean
0,AAL,721,7.952137e+06,11029.315666
1,DAL,613,6.491973e+06,10590.493908
2,SWA,546,5.940822e+06,10880.627004
3,UAL,491,5.032830e+06,10250.162381
4,RYR,405,3.391457e+06,8373.967171
...,...,...,...,...
655,SPA,1,1.131082e+04,11310.817082
656,SOL,1,7.028465e+03,7028.465472
657,SMR,1,4.666830e+03,4666.829826
658,NTA,1,8.135694e+03,8135.694048


In [26]:
print(f"Most Flight Counts: Airline = {table_sorted_count['airline'][0]} | Number of Counts = {table_sorted_count['count'][0]} \
| Distance Mean = {table_sorted_count['distance_mean'][0]}")

Most Flight Counts: Airline = AAL | Number of Counts = 721 | Distance Mean = 11029.315666015445
